In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
#url for the page
url="https://scai.engineering.asu.edu/computer-science-and-engineering-faculty/"
#adding dummy user agent to avoid getting block and access(to avoid error 403)
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
}
#request to url
page = requests.get(url, headers=headers)
soup = BeautifulSoup(page.text, 'html.parser')



In [13]:
#list to store data
Faculty_Data=[]

#opening a text file to store the scrapped data in the format of faculty name, reference link, Bio and courses taught in on horizontal line for each faculty
f=open("Name_Link.txt",'w', encoding="utf-8")
f.write("Name\t\t\t Reference link\t\t\t Bio\t\t\t Courses Taught\n\n")
try:
    for info in soup.find_all(class_="et_pb_text_inner"):
    #finding all <strong> tags
        strong_tag= info.find_all("strong")
        Faculty_Name=""
        for strong_tag in strong_tag:
            F_name=strong_tag.text.strip()
            if not F_name:
                continue
            Faculty_Name=F_name
            # nearest <p> tag to strong tag
            p_tag= strong_tag.find_parent("p")
            if p_tag:
                R_link=p_tag.find("a",href=True)
                if R_link:
                    reference_link=R_link.get("href")
                    f.write(f"{reference_link}\n")
except requests.exceptions.RequestException as e:
    pass

f.close()

In [45]:
from bs4 import BeautifulSoup
import requests



f = open("Name_Link.txt", 'w', encoding="utf-8")
f.write("Name\t\t\t Reference link\n\n")
biog = open("bio.txt", 'w', encoding="utf-8")
biog.write("Name\t\t\t BIO\n\n")
ct = open("courses.txt", 'w', encoding="utf-8")
ct.write("Name\t\t\t CoursesTaught\n\n")
Data = open("DataSet.txt", 'w', encoding="utf-8")
Data.write("Name\t\t\t Reference link\t\t\t Bio\t\t\t Courses Taught\n\n")

# Find all faculty sections with class 'post-content'
main_class = soup.find_all("div", class_="post-content")

for m_class in main_class:
    # Iterate through all the 'et_pb_text_inner' elements within the faculty section
    for info in m_class.find_all(class_="et_pb_text_inner"):
        # finding all <strong> tags
        strong_tag = info.find_all("strong")
        Faculty_Name = ""
        for strong_tag in strong_tag:
            F_name = strong_tag.text.strip()
            if not F_name:
                continue
            Faculty_Name = F_name
            # nearest <p> tag to strong tag
            p_tag = strong_tag.find_parent("p")
            if p_tag:
                R_link = p_tag.find("a", href=True)
                if R_link:
                    reference_link = R_link.get("href")
                    bio = ""
                    courses_taught = set()  # Use a set to avoid duplicates

                    try:
                        # requesting faculty homepage
                        response = requests.get(reference_link, headers=headers)
                        faculty_soup = BeautifulSoup(response.text, "html.parser")

                        # Extract the bio text
                        bio_section = faculty_soup.find("div", class_="field__item")
                        if bio_section:
                            # Finding all <p> tags and joining them to make one para
                            bio = ' '.join([p.text.strip() for p in bio_section.find_all("p")])

                        # Find all tables with class "webdir-profile-courses responsive-enabled table"
                        course_tables = faculty_soup.find_all("table", class_="webdir-profile-courses responsive-enabled table")
                        for index, course_table in enumerate(course_tables, start=1):
                            # Iterate through rows in the table
                            for row in course_table.find_all("tr"):
                                # Extract all text within the row
                                row_text = row.get_text(strip=True)
                                # Append non-empty rows to the courses_taught set
                                if row_text:
                                    courses_taught.add(row_text)

                        courses_taught_str = ", ".join(courses_taught)

                        f.write(f"Name:{F_name}\t\tLink:{reference_link}\n")
                        biog.write(f"Name:{F_name}\t\t Bio:{bio}\n")
                        ct.write(f"Name:{F_name}\t\t Course Taught:{courses_taught_str}\n")
                        Data.write(f"Name:{F_name}\t\t\t Link:{reference_link}\t\t\t Bio:{bio}\t\t\tCourse Taught:{courses_taught_str}\n")

                    except requests.exceptions.RequestException as e:
                        pass

# Close the output file
f.close()
biog.close()
ct.close()
Data.close()
